# Worksheet 05

Name: Jeff Tedi
UID: U47550540

### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

All points will be asssigned to either 0 or 2 based on its distance to these centroids.

0: 0, 0.5
2: 1.5, 2, 6, 6.5, 7

Centroid of these points: 0.25, 4.6

0.25: 0, 0.5, 1.5, 2
4.6: 6, 6.5, 7

Centroid of next iteration: 1, 6.5

1: 0, 0.5, 1.5, 2
6.5: 6, 6.5, 7

End of iteration(s)

b) Describe in plain english what the cost function for k means is.

The distances from k-clusters to its respective centroids.

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

It is because datasets are random enough that it does not necessarily converge into a particular value.

d) Does Lloyd's Algorithm always converge? Why / why not?

Yes, because it is impossible for this program to loop.

e) Follow along in class the implementation of Kmeans

In [ ]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []

    def snap(self, centers):
        TEMPFILE = "temp.png"
        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))

    def initialize(self):
        return self.data[np.random.choice(range(len(self.data)),self.k, replace = False)]
    
    def distance(self,x,y):
        return np.linalg.norm(x-y)
  
    def assign(self, centers):
        for i in range(len(self.data)):
            delta = [float('inf'),0]
            for j in range(len(centers)):
                distance = self.distance(centers[j],self.data[i])
                if distance<delta[0]:
                    delta[0] = distance 
                    delta[1] = j 
            self.assignment[i] = delta[1]
            
    def get_centers(self):
        centers = []
        for i in set(self.assignment):
            cluster = []
            for j in range(len(self.data)):
                if self.assignment[j] ==i:
                    cluster.append(self.data[j])
            x = 0
            y = 0
            for delta in range(len(cluster)):
                x+=cluster[delta][0]
                y+=cluster[delta][1]
            centers.append([x/len(cluster), y/len(cluster)])
        return np.array(centers)
            
    def is_diff_centers(self,centers, new_centers):
        n = len(centers)
        flag = 0
        for i in range(n):
            if centers[i][0]!=new_centers[i][0]:
                flag = 1
        if flag ==1:
            return True 
        return False

    def lloyds(self):
        centers = self.initialize()
        self.assign(centers)
        self.snap(centers)
        new_centers = self.get_centers()
        while self.is_diff_centers(centers,new_centers):
            self.assign(new_centers)
            centers = new_centers
            self.snap(centers)
            new_centers = self.get_centers()
        return  

kmeans = KMeans(X, 4)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)